# AlexNet
2012年，AlexNet横空出世。这个模型的名字来源于论文第一作者的姓名Alex Krizhevsky [1]。AlexNet使用了8层卷积神经网络，并以很大的优势赢得了ImageNet 2012图像识别挑战赛。它首次证明了学习到的特征可以超越手工设计的特征，从而一举打破计算机视觉研究的前状。

AlexNet与LeNet的设计理念非常相似，但也有显著的区别。

第一，与相对较小的LeNet相比，AlexNet包含8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。下面我们来详细描述这些层的设计。

AlexNet第一层中的卷积窗口形状是$11\times11$。因为ImageNet中绝大多数图像的高和宽均比MNIST图像的高和宽大10倍以上，ImageNet图像的物体占用更多的像素，所以需要更大的卷积窗口来捕获物体。第二层中的卷积窗口形状减小到$5\times5$，之后全采用$3\times3$。此外，第一、第二和第五个卷积层之后都使用了窗口形状为$3\times3$、步幅为2的最大池化层。而且，AlexNet使用的卷积通道数也大于LeNet中的卷积通道数数十倍。

紧接着最后一个卷积层的是两个输出个数为4,096的全连接层。这两个巨大的全连接层带来将近1 GB的模型参数。由于早期显存的限制，最早的AlexNet使用双数据流的设计使一块GPU只需要处理一半模型。幸运的是，显存在过去几年得到了长足的发展，因此通常我们不再需要这样的特别设计了。

第二，AlexNet将sigmoid激活函数改成了更加简单的ReLU激活函数。一方面，ReLU激活函数的计算更简单，例如它并没有sigmoid激活函数中的求幂运算。另一方面，ReLU激活函数在不同的参数初始化方法下使模型更容易训练。这是由于当sigmoid激活函数输出极接近0或1时，这些区域的梯度几乎为0，从而造成反向传播无法继续更新部分模型参数；而ReLU激活函数在正区间的梯度恒为1。因此，若模型参数初始化不当，sigmoid函数可能在正区间得到几乎为0的梯度，从而令模型无法得到有效训练。

第三，AlexNet通过丢弃法（参见“丢弃法”一节）来控制全连接层的模型复杂度。而LeNet并没有使用丢弃法。

第四，AlexNet引入了大量的图像增广，如翻转、裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合。

In [1]:
### Publised by ClearTourch on June 18 ,2020 ###

import tensorflow as tf
from tensorflow.keras import layers, models, losses
import numpy as np

print('TF version:', tf.__version__)
print('Numpy version:', np.__version__)
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

TF version: 2.2.0
Numpy version: 1.18.1


In [3]:
class AlexNet:
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0,axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0,axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]

    def build_alexnet(self):
        net = tf.keras.models.Sequential([
            layers.Conv2D(filters=96,kernel_size=11,strides=4, input_shape=(224, 224, 1),
                                   activation='relu'),
            layers.MaxPool2D(pool_size=3, strides=2),
            layers.Conv2D(filters=256,kernel_size=5,padding='same',activation='relu'),
            layers.MaxPool2D(pool_size=3, strides=2),
            layers.Conv2D(filters=384,kernel_size=3,padding='same',activation='relu'),
            layers.Conv2D(filters=384,kernel_size=3,padding='same',activation='relu'),
            layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'),
            layers.MaxPool2D(pool_size=3, strides=2),
            layers.Flatten(),
            layers.Dense(4096,activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(4096,activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(10,activation='sigmoid')
        ])
        net.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
        #net.summary()
        return net

    def see_output_shape(self):
        x = tf.random.uniform((1, 224, 224, 1))
        for layer in alexnet().layers:
            x = layer(x)
            print(layer.name, 'output shape\t', x.shape)
            
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.train_images[index],224,224,)
        return resized_images.numpy(), self.train_labels[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.test_images[index],224,224,)
        return resized_images.numpy(), self.test_labels[index]
    
    def train_alexnet(self, net, batch_size):
        epoch = 5
        num_iter = AlexNet().num_train//batch_size
        for e in range(epoch):
            for n in range(num_iter):
                x_batch, y_batch = dataLoader.get_batch_train(batch_size)
                net.fit(x_batch, y_batch)
                if n%20 == 0:
                    net.save_weights("./ModelTrain/alexnet_weights.h5")


In [4]:
if __name__ == '__main__':
    batch_size = 128
    alexnet = AlexNet()
    net = alexnet.build_alexnet()
    x_batch, y_batch = alexnet.get_batch_train(batch_size)
    print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)
    alexnet.train_alexnet(net, batch_size)
    
    net.load_weights("./ModelTrain/alexnet_weights.h5")

    x_test, y_test = alexnet.get_batch_test(2000)
    net.evaluate(x_test, y_test, verbose=2)


NameError: name 'optimizer' is not defined